**Mount_SDcard_with_BBB.md**
- Mounting SD card is not so easy with BBB, it is due to the problem of mysql error, saying sock file is missing and while starting satying /var folder is full
- By cheking ```df -h``` shows it is true. So only way to increase the partition is use sd card, but noraml sd card with partiion cannot mount with BBB, due to some kernal inclination
- there is work arroun, based on [[this]](http://dev.gentoo.org/~armin76/arm/beagleboneblack/install_emmc.xml) under title code listing 7.2, fromatting microsd card
- wget http://dev.gentoo.org/~armin76/arm/beaglebone/mkcard.sh `bash mkcard.sh /dev/mmcblk0`
- We need to pass an special environment to U-Boot, for doing so we need to create a file called uEnv.txt with the following contents on it.
```
Code Listing 7.3: uEnv.txt contents
bootpart=1:2
mmcroot=/dev/mmcblk1p2
Mount the first partition of the microSD card and copy this uEnv.txt file
Code Listing 7.4: Copying U-Boot and MLO to the microSD card
(Make sure you're root)
(Replace mmcblk0p1 with sdd1 if your card showed up as /dev/sdd)
# mkdir /mnt/p1 ; mount /dev/mmcblk0p1 /mnt/p1
# cp uEnv.txt /mnt/p1
(Unmount the partition)
# umount /mnt/p1
```
1. The contents
```bash
#! /bin/sh
# mkcard.sh v0.5
# (c) Copyright 2009 Graeme Gregory <dp@xora.org.uk>
# Licensed under terms of GPLv2
#
# Parts of the procudure base on the work of Denys Dmytriyenko
# http://wiki.omap.com/index.php/MMC_Boot_Format
export LC_ALL=C
if [ -z `which bc` ]; then
        echo "no bc binary found"
        exit 1;
fi
if [ $# -ne 1 ]; then
        echo "Usage: $0 <drive>"
        exit 1;
fi
DRIVE=$1
dd if=/dev/zero of=$DRIVE bs=1024 count=1024
SIZE=`fdisk -l $DRIVE | grep Disk | grep bytes | awk '{print $5}'`
echo DISK SIZE - $SIZE bytes=
CYLINDERS=`echo $SIZE/255/63/512 | bc`
echo CYLINDERS - $CYLINDERS
{
echo ,9,0x0C,*
echo ,,,-
} | sfdisk -D -H 255 -S 63 -C $CYLINDERS $DRIVE
```